# IMPORTING LIBRARIES TO LOAD DATA

In [83]:
#importing libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [84]:
#loading the data set train and test
train = pd.read_csv("train.csv")

In [85]:
print("The shape of the train data is :", train.shape)

The shape of the train data is : (1168, 81)


In [86]:
#checking the info of the train data
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1168 entries, 0 to 1167
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1168 non-null   int64  
 1   MSSubClass     1168 non-null   int64  
 2   MSZoning       1168 non-null   object 
 3   LotFrontage    954 non-null    float64
 4   LotArea        1168 non-null   int64  
 5   Street         1168 non-null   object 
 6   Alley          77 non-null     object 
 7   LotShape       1168 non-null   object 
 8   LandContour    1168 non-null   object 
 9   Utilities      1168 non-null   object 
 10  LotConfig      1168 non-null   object 
 11  LandSlope      1168 non-null   object 
 12  Neighborhood   1168 non-null   object 
 13  Condition1     1168 non-null   object 
 14  Condition2     1168 non-null   object 
 15  BldgType       1168 non-null   object 
 16  HouseStyle     1168 non-null   object 
 17  OverallQual    1168 non-null   int64  
 18  OverallC

In [87]:
#droping the ID column from train and test dataset,#there is only 1 value in train for utilities, so it will have no variance
train.drop(['Id','Utilities'],inplace=True,axis=1)

# MISSING VALUE ANALYSIS

In [88]:
#finding the missing values for both train and test data set
#let's see the percentage of missing values in each column.
train_null = train.isnull().sum()/train.shape[0]*100
train_null

MSSubClass        0.000000
MSZoning          0.000000
LotFrontage      18.321918
LotArea           0.000000
Street            0.000000
Alley            93.407534
LotShape          0.000000
LandContour       0.000000
LotConfig         0.000000
LandSlope         0.000000
Neighborhood      0.000000
Condition1        0.000000
Condition2        0.000000
BldgType          0.000000
HouseStyle        0.000000
OverallQual       0.000000
OverallCond       0.000000
YearBuilt         0.000000
YearRemodAdd      0.000000
RoofStyle         0.000000
RoofMatl          0.000000
Exterior1st       0.000000
Exterior2nd       0.000000
MasVnrType        0.599315
MasVnrArea        0.599315
ExterQual         0.000000
ExterCond         0.000000
Foundation        0.000000
BsmtQual          2.568493
BsmtCond          2.568493
BsmtExposure      2.654110
BsmtFinType1      2.568493
BsmtFinSF1        0.000000
BsmtFinType2      2.654110
BsmtFinSF2        0.000000
BsmtUnfSF         0.000000
TotalBsmtSF       0.000000
H

***I am filliing up the missing values of Lot Frontage as the mean value of Lot Frontage grouped by Lot Config***

In [89]:
train_copy = train.copy()

for var_class in train['LotConfig'].unique():

    train_copy.update(train[train.loc[:,'LotConfig']==var_class]['LotFrontage'].replace(np.nan,train[train.loc[:,'LotConfig']==var_class]['LotFrontage'].median()))

In [90]:
#analysing alley column
# train['Alley'].unique()
# I will replace the Nan value with no

train_copy['Alley'].fillna('none',inplace=True)
train_copy['Alley'].isnull().sum()

0

In [91]:
#analyzing MasVnrArea and MasVnrType
train_copy[train_copy['MasVnrArea'].isnull()][['MasVnrType','MasVnrArea']]

MasVnrType  MasVnrArea
68         NaN         NaN
78         NaN         NaN
99         NaN         NaN
185        NaN         NaN
224        NaN         NaN
367        NaN         NaN
874        NaN         NaN

In [92]:
#I will be taking all the rows where MasVnrType is not null
train_copy = train_copy[train_copy['MasVnrType'].notna()]
train_copy.shape

(1161, 79)

In [93]:
#analysing basement related columns
#there are null values in some columns, I will be replacing them as per the data description
for i in ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']:
    print("Unique values in {} are {}".format(i,train_copy[i].unique()))

Unique values in BsmtQual are ['Gd' 'TA' 'Ex' nan 'Fa']
Unique values in BsmtCond are ['TA' 'Gd' 'Fa' nan 'Po']
Unique values in BsmtExposure are ['No' 'Gd' 'Av' 'Mn' nan]
Unique values in BsmtFinType1 are ['ALQ' 'GLQ' 'BLQ' 'Unf' 'Rec' 'LwQ' nan]
Unique values in BsmtFinType2 are ['Unf' 'Rec' 'BLQ' 'GLQ' nan 'ALQ' 'LwQ']


as per the data description 'na' in ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2'] means 'no basement', so I will replace the na with No_base

In [94]:
for i in ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']:
    train_copy[i].fillna('none',inplace=True)

In [95]:
for i in ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']:
    print("Unique values in train {} are {}".format(i,train_copy[i].unique()))
    print('='*50)

Unique values in train BsmtQual are ['Gd' 'TA' 'Ex' 'none' 'Fa']
Unique values in train BsmtCond are ['TA' 'Gd' 'Fa' 'none' 'Po']
Unique values in train BsmtExposure are ['No' 'Gd' 'Av' 'Mn' 'none']
Unique values in train BsmtFinType1 are ['ALQ' 'GLQ' 'BLQ' 'Unf' 'Rec' 'LwQ' 'none']
Unique values in train BsmtFinType2 are ['Unf' 'Rec' 'BLQ' 'GLQ' 'none' 'ALQ' 'LwQ']


In [96]:
#cleaning the FirplaceQu column
train_copy['FireplaceQu'].fillna('none',inplace=True)
print("Unique values in train Fire place quality is", train_copy['FireplaceQu'].unique())

Unique values in train Fire place quality is ['TA' 'Gd' 'none' 'Fa' 'Ex' 'Po']


In [97]:
for i in ['GarageQual','GarageCond','GarageType','GarageFinish']:
    print("Unique values in {} are {}".format(i,train_copy[i].unique()))

Unique values in GarageQual are ['TA' 'Fa' nan 'Gd' 'Ex' 'Po']
Unique values in GarageCond are ['TA' 'Fa' 'Gd' nan 'Po' 'Ex']
Unique values in GarageType are ['Attchd' 'BuiltIn' 'Detchd' 'Basment' nan '2Types' 'CarPort']
Unique values in GarageFinish are ['RFn' 'Unf' 'Fin' nan]


as per the data description 'nan' value in  ['GarageQual','GarageCond'] means that there are 'no Garage', I will replace the nan with No_Garage

In [98]:
for i in ['GarageQual','GarageCond','GarageType','GarageFinish']:
    train_copy[i].fillna('none',inplace=True)

In [99]:
for i in ['GarageQual','GarageCond','GarageType','GarageFinish']:
    print("Unique values in {} are {}".format(i,train_copy[i].unique()))
    print('='*50)

Unique values in GarageQual are ['TA' 'Fa' 'none' 'Gd' 'Ex' 'Po']
Unique values in GarageCond are ['TA' 'Fa' 'Gd' 'none' 'Po' 'Ex']
Unique values in GarageType are ['Attchd' 'BuiltIn' 'Detchd' 'Basment' 'none' '2Types' 'CarPort']
Unique values in GarageFinish are ['RFn' 'Unf' 'Fin' 'none']


In [100]:
#cleaning PoolQC column
train_copy['PoolQC'].fillna('none',inplace=True)

In [101]:
#cleaning Fence column
train_copy['Fence'].fillna('none',inplace=True)

In [102]:
#cleaning MiscFeature column
train_copy['MiscFeature'].fillna('none',inplace=True)

In [103]:
print(train_copy.shape)

(1161, 79)


In [104]:
#as per data description, I am changing C (all) to C only
train_copy['MSZoning'].replace({'C (all)':'C'},inplace=True)

In [105]:
train_copy['MSZoning'].unique()

array(['RL', 'RM', 'FV', 'RH', 'C'], dtype=object)

In [106]:
#checking columns having missing values
nan_var_train = train_copy.isnull().sum()/train_copy.shape[0]*100
nan_var_train[nan_var_train>0]

GarageYrBlt    5.512489
dtype: float64

In [107]:
train_copy.to_csv('cleaned_train.csv')

In [108]:
train_copy.head()

MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0         120       RL         68.0     4928   Pave  none      IR1   
1          20       RL         95.0    15865   Pave  none      IR1   
2          60       RL         92.0     9920   Pave  none      IR1   
3          20       RL        105.0    11751   Pave  none      IR1   
4          20       RL         65.0    16635   Pave  none      IR1   

  LandContour LotConfig LandSlope Neighborhood Condition1 Condition2 BldgType  \
0         Lvl    Inside       Gtl      NPkVill       Norm       Norm   TwnhsE   
1         Lvl    Inside       Mod        NAmes       Norm       Norm     1Fam   
2         Lvl   CulDSac       Gtl      NoRidge       Norm       Norm     1Fam   
3         Lvl    Inside       Gtl       NWAmes       Norm       Norm     1Fam   
4         Lvl       FR2       Gtl       NWAmes       Norm       Norm     1Fam   

  HouseStyle  OverallQual  OverallCond  YearBuilt  YearRemodAdd RoofStyle  \
0     1Story            6            5       1976          1976     Gable   
1     1Story            8            6       1970          1970      Flat   
2     2Story            7            5       1996          1997     Gable   
3     1Story            6            6       1977          1977       Hip   
4     1Story            6            7       1977          2000     Gable   

  RoofMatl Exterior1st Exterior2nd MasVnrType  MasVnrArea ExterQual ExterCond  \
0  CompShg     Plywood     Plywood       None         0.0        TA        TA   
1  Tar&Grv     Wd Sdng     Wd Sdng       None         0.0        Gd        Gd   
2  CompShg     MetalSd     MetalSd       None         0.0        Gd        TA   
3  CompShg     Plywood     Plywood    BrkFace       480.0        TA        TA   
4  CompShg     CemntBd     CmentBd      Stone       126.0        Gd        TA   

  Foundation BsmtQual BsmtCond BsmtExposure BsmtFinType1  BsmtFinSF1  \
0     CBlock       Gd       TA           No          ALQ         120   
1      PConc       TA       Gd           Gd          ALQ         351   
2      PConc       Gd       TA           Av          GLQ         862   
3     CBlock       Gd       TA           No          BLQ         705   
4     CBlock       Gd       TA           No          ALQ        1246   

  BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF Heating HeatingQC  \
0          Unf           0        958         1078    GasA        TA   
1          Rec         823       1043         2217    GasA        Ex   
2          Unf           0        255         1117    GasA        Ex   
3          Unf           0       1139         1844    GasA        Ex   
4          Unf           0        356         1602    GasA        Gd   

  CentralAir Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  \
0          Y      SBrkr       958         0             0        958   
1          Y      SBrkr      2217         0             0       2217   
2          Y      SBrkr      1127       886             0       2013   
3          Y      SBrkr      1844         0             0       1844   
4          Y      SBrkr      1602         0             0       1602   

   BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  \
0             0             0         2         0             2             1   
1             1             0         2         0             4             1   
2             1             0         2         1             3             1   
3             0             0         2         0             3             1   
4             0             1         2         0             3             1   

  KitchenQual  TotRmsAbvGrd Functional  Fireplaces FireplaceQu GarageType  \
0          TA             5        Typ           1          TA     Attchd   
1          Gd             8        Typ           1          TA     Attchd   
2          TA             8        Typ           1          TA     Attchd   
3          TA             7        Typ           1 